# Подгонка ARMA (пакет [`statsmodels`](https://www.statsmodels.org/stable/index.html))

__Важно__: $ARMA$ рассматривается как частный случай $ARIMA$: 

$$
	ARMA(p,q)=ARIMA(p,0,q).
$$

Для подгонки используем классы 
* [ARIMA](https://www.statsmodels.org/stable/generated/statsmodels.tsa.arima.model.ARIMA.html#statsmodels.tsa.arima.model.ARIMA) (специфицированная модель)
* [ARIMAResult](https://www.statsmodels.org/stable/generated/statsmodels.tsa.arima.model.ARIMAResults.html#statsmodels.tsa.arima.model.ARIMAResults) (подогнанная модель)

In [ ]:
import numpy as np
import pandas as pd

import statsmodels.tsa.api as tsa
from statsmodels.stats.diagnostic import het_arch, acorr_ljungbox

import pandas_datareader.data as web

# настройки визуализация
import matplotlib.pyplot as plt

# Не показывать Warnings
import warnings
warnings.simplefilter(action='ignore', category=Warning)

## Подгонка ARMA заданного порядка

Загрузим из БД [`FRED`](https://fred.stlouisfed.org/) недельные данные по Market Yield on U.S. Treasury Securities at 10-Year Constant Maturity (Symbol [`WGS10YR`](https://fred.stlouisfed.org/series/WGS10YR)) с 2000-01-01 по 2023-12-31 и создадим датафрейм `rate`



In [ ]:
rate = web.DataReader(name='WGS10YR', data_source='fred', start='2000-01-01', end='2023-12-31')

Пусть `y` – первая разность ставки (не забудем удалить пропущенные наблюдения)

In [ ]:
y = rate.diff().dropna()

Подгоним модель ARMA(2,1) для `y`

Спецификация

$$
	y_t=\alpha_0+\phi_1y_{t-1}+\phi_2y_{t-2}+u_1+\theta u_{t-1}
$$

In [ ]:
# спецификация модели
arma_mod = tsa.ARIMA(y, order=(2,0,1), trend='c', missing='drop')
# подгонка модели на данных
arma_res = arma_mod.fit()
# выводим результаты подгонки
arma_res.summary(alpha=0.05)

## Диагностика подогнанной модели

Базовые графики

In [ ]:
arma_res.plot_diagnostics(lags=15)

plt.show()

Тест на серийную корреляцию (Ljung-Box test). Выберем число лагов 10

In [ ]:
arma_res.test_serial_correlation(method='ljungbox', lags=[10], df_adjust=True)

Альтернативный вариант

In [ ]:
acorr_ljungbox(arma_res.resid, lags=[10], model_df=2+1)

Тест на гетероскедастичность (на ARCH-эффекты)

In [ ]:
lm_stat, lm_pval, f_stat, f_pval = het_arch(arma_res.resid, nlags=7, ddof=2+1)

lm_stat, lm_pval

## Подбор порядка лагов

Используем информационные критерии для подбора "оптимального" порядка модели:
* BIC
* AIC
* AICC
* HQIC

__Замечаение__: результат выдаётся в виде словаря

In [ ]:
# используем критерий BIC
tsa.arma_order_select_ic(y, max_ar=4, max_ma=4, trend='c', ic='bic')

In [ ]:
# выведем только порядок оптимальной модели
optimal_order = tsa.arma_order_select_ic(y, max_ar=4, max_ma=4, trend='c', ic='bic')
optimal_order['bic_min_order']

In [ ]:
# Используем информационные критерии AIC, AICC, BIC, HQIC
tsa.arma_order_select_ic(y, max_ar=4, max_ma=4, trend='c', ic=['aic', 'aicc', 'bic', 'hqic'])

## Подгонка ARMA(1,3) с трендом

Спецификация модели

$$
	y_t=\alpha_0+\alpha_1t+\phi y_{t-1}+u_t+\theta_1u_{t-1}+\theta_2u_{t-2}+\theta_3u_{t-3}
$$

In [ ]:
# спецификация модели
arma_trend_mod = tsa.ARIMA(y, order=(1,0,3), trend='ct', missing='drop')
# подгонка модели на данных
arma_trend_res = arma_trend_mod.fit()
# выводим результаты подгонки
arma_trend_res.summary(alpha=0.05)